## Cointegration

In [33]:
%matplotlib inline
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt

import tushare as ts
import statsmodels.formula.api as sm
import statsmodels.tsa.stattools as smts
from statsmodels.graphics.api import qqplot

In [97]:
df_ldkg = ts.get_hist_data('600606',start='2014-12-23',end='2016-12-23')
print df_ldkg.shape
df_sh = ts.get_hist_data('sh',start='2014-12-23',end='2016-12-23')
print df_sh.shape
df_sh['stamp']= df_sh.index
df_ldkg['stamp']= df_ldkg.index
df_m=pd.merge(df_sh[['close','stamp']],df_ldkg[['close','stamp']],left_on='stamp',right_on='stamp')# merge
print df_m.shape

(469, 14)
(490, 13)
(469, 3)


In [98]:
df_m.iloc[0:5]

,close_x,stamp,close_y
0,3110.15,2016-12-23,8.87
1,3139.56,2016-12-22,9.01
2,3137.43,2016-12-21,9.06
3,3102.88,2016-12-20,9.02
4,3118.08,2016-12-19,9.02


## integral degree

<http://statsmodels.sourceforge.net/0.5.0/generated/statsmodels.tsa.stattools.adfuller.html>

In [99]:
smts.adfuller(df_m['close_x'])

(-1.7743214924586754,
 0.39328280293152296,
 15,
 453,
 {'1%': -3.4448681948588309,
  '10%': -2.5701797149735146,
  '5%': -2.867941416548597},
 5152.7164386358618)

In [82]:
smts.adfuller(df_m['close_y'])

(-2.2840674676753436,
 0.17715802862045499,
 17,
 451,
 {'1%': -3.444932949082776,
  '10%': -2.5701948966327599,
  '5%': -2.8679698999537262},
 1142.9380210216552)

In [100]:
diff_sh = df_m['close_x'].values[:-1] - df_m['close_x'].values[1:]
smts.adfuller(diff_sh)

(-5.1219524492479565,
 1.2661933836148604e-05,
 14,
 453,
 {'1%': -3.4448681948588309,
  '10%': -2.5701797149735146,
  '5%': -2.867941416548597},
 5143.455689145082)

In [101]:
diff_ldkg = df_m['close_y'].values[:-1] - df_m['close_y'].values[1:]
smts.adfuller(diff_ldkg)

(-4.0832213616888478,
 0.0010322568520231509,
 16,
 451,
 {'1%': -3.444932949082776,
  '10%': -2.5701948966327599,
  '5%': -2.8679698999537262},
 1144.7881122175286)

>The first order difference of the two price series are both not unit root.

## regression => cointegration coef

In [102]:
dsf=pd.Series(diff_sh,
          index = df_m['stamp'][:-1]) 
dsf.shape

(468,)

In [103]:
dldkg=pd.Series(diff_ldkg,
          index = df_m['stamp'][:-1]) 
dldkg.shape

(468,)

In [109]:
df_mm = pd.DataFrame({'shDiff':dsf , 'ldkgDiff':dldkg})


In [111]:
result = sm.ols(formula="shDiff ~  ldkgDiff", data=df_mm).fit()
print result.params

Intercept     0.490954
ldkgDiff     56.172921
dtype: float64


In [112]:
print result.summary()

                            OLS Regression Results                            
Dep. Variable:                 shDiff   R-squared:                       0.418
Model:                            OLS   Adj. R-squared:                  0.417
Method:                 Least Squares   F-statistic:                     335.2
Date:                Sat, 24 Dec 2016   Prob (F-statistic):           8.20e-57
Time:                        03:10:57   Log-Likelihood:                -2560.5
No. Observations:                 468   AIC:                             5125.
Df Residuals:                     466   BIC:                             5133.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.4910      2.665      0.184      0.8